# Sampling and Splitting

## Sampling

Sampling for basic tabular datasets. (Not designed for time series as of now.)

In [1]:
import polars as pl
import polars_ds as pds
import polars_ds.sample as sa

In [2]:
df = pds.frame(size=100_000).with_columns(
    pds.random(0.0, 12.0).alias("uniform_1"),
    pds.random(0.0, 1.0).alias("uniform_2"),
    pds.random_exp(0.5).alias("exp"),
    pds.random_normal(0.0, 1.0).alias("normal"),
    pds.random_normal(0.0, 1000.0).alias("fat_normal"),
    (pds.random_int(0, 3)).alias("flags"),
    pl.Series(["A"] * 30_000 + ["B"] * 30_000 + ["C"] * 40_000).alias("category"),
)
df.head()

row_num,uniform_1,uniform_2,exp,normal,fat_normal,flags,category
i64,f64,f64,f64,f64,f64,i32,str
0,1.442612,0.723444,0.448733,-0.084833,-1056.526036,2,"""A"""
1,4.989622,0.015146,0.903302,2.962368,-1014.483108,1,"""A"""
2,9.686805,0.21509,0.835874,0.877411,-1273.97176,1,"""A"""
3,3.613803,0.14252,0.002497,0.359183,-428.85816,1,"""A"""
4,1.856903,0.732123,1.876483,0.168933,1385.691192,0,"""A"""


In [3]:
sa.random_cols(df, 2, keep = ["row_num"])

['row_num', 'uniform_2', 'fat_normal']

In [4]:
# Random Sample
sa.sample(df, 0.6) # by ratio

row_num,uniform_1,uniform_2,exp,normal,fat_normal,flags,category
i64,f64,f64,f64,f64,f64,i32,str
0,1.442612,0.723444,0.448733,-0.084833,-1056.526036,2,"""A"""
1,4.989622,0.015146,0.903302,2.962368,-1014.483108,1,"""A"""
3,3.613803,0.14252,0.002497,0.359183,-428.85816,1,"""A"""
4,1.856903,0.732123,1.876483,0.168933,1385.691192,0,"""A"""
6,5.811808,0.989879,1.489097,0.094162,-517.926376,1,"""A"""
…,…,…,…,…,…,…,…
99990,6.656671,0.611289,6.087972,-0.250988,-56.640158,0,"""C"""
99992,3.166684,0.60644,2.020982,-0.505243,1162.715874,1,"""C"""
99994,6.969484,0.37924,0.32146,-0.078272,-917.49053,0,"""C"""


In [5]:
sa.sample(df, 30_000) # by count

row_num,uniform_1,uniform_2,exp,normal,fat_normal,flags,category
i64,f64,f64,f64,f64,f64,i32,str
3,3.613803,0.14252,0.002497,0.359183,-428.85816,1,"""A"""
4,1.856903,0.732123,1.876483,0.168933,1385.691192,0,"""A"""
7,10.586623,0.518819,9.577112,1.402459,-1194.209391,0,"""A"""
9,7.851517,0.178806,3.892864,-1.331864,221.991515,2,"""A"""
10,6.793582,0.507613,0.766305,0.999309,-71.046855,2,"""A"""
…,…,…,…,…,…,…,…
99982,5.376131,0.497036,0.366662,0.260872,564.002351,1,"""C"""
99986,1.101901,0.986311,0.638334,0.965154,2246.819502,1,"""C"""
99988,8.57487,0.283489,2.209421,0.003966,1417.797346,0,"""C"""


In [6]:
df.group_by("flags").len().sort("flags")

flags,len
i32,u32
0,33541
1,33270
2,33189


In [7]:
# Downsample on one group
sa1 = sa.downsample(
    df, 
    (pl.col("flags") == 0, 0.5)
)
sa1.group_by("flags").len().sort("flags")

flags,len
i32,u32
0,16771
1,33270
2,33189


In [8]:
# Downsample on multiple groups
sa2 = sa.downsample(
    df, 
    (pl.col("flags") == 0, 0.5),
    (pl.col("flags") == 1, 0.3),
    (pl.col("flags") == 2, 0.4),
)
sa2.group_by("flags").len().sort("flags")

flags,len
i32,u32
0,16771
1,9981
2,13276


In [9]:
df.group_by("category").len().sort("category")

category,len
str,u32
"""A""",30000
"""B""",30000
"""C""",40000


In [10]:
# Volume neutral by each category, will take the greatest possible value so that we get neutral volume.
vn = sa.volume_neutral(
    df,
    by = pl.col("category"),
)
vn.group_by("category").len().sort("category")

category,len
str,u32
"""A""",30000
"""B""",30000
"""C""",30000


In [11]:
# Volume neutral (10_000) by each category
vn = sa.volume_neutral(
    df,
    by = pl.col("category"),
    target_volume = 10_000
)
vn.group_by("category").len().sort("category")

category,len
str,u32
"""A""",10000
"""B""",10000
"""C""",10000


In [12]:
# Volume neutral (10_000) by a more complicated condition
vn = sa.volume_neutral(
    df,
    by = pl.col("category") == "A",
    target_volume = 10_000
) # This makes sense because count for B + count for C = 10_000
vn.group_by("category").len().sort("category")

category,len
str,u32
"""A""",10000
"""B""",4283
"""C""",5717


In [13]:
# Volume neutral sample with a control level. Volume neutral happens under the category level, meaning
# the volume for each flag in each category is neutral.
vn = sa.volume_neutral(
    df,
    by = pl.col("flags"),
    control = pl.col("category")
) 
vn.group_by(["category", "flags"]).len().sort(["category", "flags"])

category,flags,len
str,i32,u32
"""A""",0,9932
"""A""",1,9932
"""A""",2,9932
"""B""",0,9942
"""B""",1,9942
"""B""",2,9942
"""C""",0,13283
"""C""",1,13283
"""C""",2,13283


In [14]:
# We may not meet the target volume for all categories.
vn = sa.volume_neutral(
    df,
    by = pl.col("flags"),
    control = pl.col("category"),
    target_volume= 10_000
) # 
vn.group_by(["category", "flags"]).len().sort(["category", "flags"])

category,flags,len
str,i32,u32
"""A""",0,9932
"""A""",1,9932
"""A""",2,9932
"""B""",0,9942
"""B""",1,9942
"""B""",2,9942
"""C""",0,10000
"""C""",1,10000
"""C""",2,10000


## Splitting

To be added...